## タスクごとのモデル比較

このノートブックでは、Mistral-7Bと並行して別のモデルであるFlan-T5-largeを使用し、その動作を確認します。

Flan-T5-Largeは確かに小型で、GPUを使用せず、4GBのRAMだけで動作しますが、タスクに対応できるでしょうか？

### 必要なものとインポート

もしLabの指示通りに適切なワークベンチイメージを選択して起動していれば、必要なライブラリはすでに揃っているはずです。そうでない場合は、次のセルの最初の行をアンコメントして、すべての必要なパッケージをインストールしてください。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 適切なワークベンチイメージを選択していない場合のみアンコメントしてください

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI, HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchainパイプライン

ここで、2つの異なるLLMエンドポイントと2つの異なるLangchainパイプラインを定義します。

In [ ]:
# LLM推論サーバーURL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLM定義
llm = VLLMOpenAI(           # vLLMのOpenAI互換APIクライアントを使用しますが、モデルはOpenShift上で動作しており、OpenAIではありません。
    openai_api_key="EMPTY",   # そのため、OpenAIキーは不要です。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Flan-T5-Small LLM推論サーバーURL
inference_server_url_flan_t5 = "http://llm-flant5.ic-shared-llm.svc.cluster.local:3000/"

# LLM定義
llm_flant5 = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_flan_t5,
    max_new_tokens=96,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

両モデルに対する**テンプレート**は同じものを使用します。

In [ ]:
template="""<s>[INST]
あなたは助けになる、丁寧で正直なアシスタントです。
常に慎重に、尊敬と真実をもって対応してください。
有用でありつつも、安全な応答を心がけてください。
有害、非倫理的、偏見のある、または否定的な内容を避け、公正さと前向きな応答を心がけてください。
私はテキストを提示し、その後に質問をします。この質問に対して、できる限り簡潔かつ正確に答えてください。

### テキスト:
{text}

### 質問:
{query}

### 答え:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

これで、2つの**会話**オブジェクトを作成し、それらを使用して2つのモデルにクエリを送ることができます。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )
conversation_flant5 = LLMChain(llm=llm_flant5,
                        prompt=PROMPT,
                        verbose=False
                        )

さあ、モデルにクエリを送る準備が整いました！

この例では、1つのクレームに対してのみクエリを送り、その結果を確認します。もちろん、異なるクレームで試してみても構いません。

In [ ]:
# クレーム
text = "蜂蜜は、最も古い自然の甘味料であると考えられています。"

# 質問
query = "蜂蜜が最も古い自然の甘味料であると考えられている理由は何ですか？"

# 両モデルへのクエリ
conversation.run(text=text, query=query)
conversation_flant5.run(text=text, query=query)